In [148]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

In [149]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

In [150]:
import findspark
findspark.init()

In [151]:
from pyspark.sql import SparkSession

def init_spark(app_name="HelloWorldApp", execution_mode="local[*]"):
  spark = SparkSession.builder.master(execution_mode).appName(app_name).getOrCreate()
  sc = spark.sparkContext
  return spark, sc

In [198]:
def main1():
  _, sc = init_spark()
  nums = sc.parallelize([1, 2, 3, 4])
  print(nums.map(lambda x: x*x).collect())

if __name__ == '__main__':
  main1()

[1, 4, 9, 16]


In [ ]:

from graphframes import GraphFrame
import pyspark.sql.functions as F 

df = spark.read.text("edges.txt")
split_col = f.split(df['value'], '	')
df = df.withColumn('src', split_col.getItem(0))
df = df.withColumn('dst', split_col.getItem(1))
edges = df.drop("value")
# edges.show()

df = spark.read.text("Vertex.txt")
split_col = f.split(df['value'], '	')
df = df.withColumn('id', split_col.getItem(0))
df = df.withColumn('name', split_col.getItem(1))
vertex =  df.drop("value")
# g  = GraphFrame(vertex , edges)

dstcount =  edges.groupBy("dst").count().select('dst' , F.col('count').alias('degree'))
srccount =  edges.groupBy("src").count().select('src' , F.col('count').alias('degree'))



dstcount.orderBy('degree' , ascending=False).show()
srccount.orderBy('degree' , ascending=False).show()



In [ ]:
CC = g.connectedComponents()
CC.select("id", "component").orderBy("component").show()


In [197]:

bestarticle = dstcount.orderBy('degree' , ascending=False)
joinresult = bestarticle.join(vertex,vertex.id ==  bestarticle.dst,"inner")
joinresult.orderBy('degree' , ascending=False).show()


+-------------------+------+-------------------+--------------------+
|                dst|degree|                 id|                name|
+-------------------+------+-------------------+--------------------+
| 946065507707541358|   327| 946065507707541358| The Football League|
|3856212023725725593|   322|3856212023725725593|National Football...|
|8978262722425160811|   316|8978262722425160811|Australian Footba...|
|6245498229508734555|   185|6245498229508734555|Southern Football...|
|7264519433548233535|   180|7264519433548233535|Football League F...|
|5362090331808156011|   179|5362090331808156011|Football League S...|
| 277710621679830671|   149| 277710621679830671|All-Ireland Senio...|
|1984578398767042266|   145|1984578398767042266|Scottish Football...|
|2395551540800395672|   134|2395551540800395672|Football League T...|
|5395033957924805072|   130|5395033957924805072|Pro Football Hall...|
|4254821691068011447|   119|4254821691068011447| Football League Two|
|7050959159375889025